# 앙상블(머신러닝)
```
앙상블 학습은 여러 개의 분류기를 생성하고, 그 예측을 결합함으로써 보다 정확한 예측을 도출하는 기법

강력한 하나의 모델을 사용하는 대신 보다 약한 모델을 여러 개를 조합하여 더 정확한 예측에 도움을 주는 방식
```

# 앙상블 학습 유형
```
- 보팅(Voting)
여러 개의 분류기가 투표를 통해 최종 예측 결과를 결정하는 방식
서로 다른 알고리즘을 여러 개 결합
하드 보팅과 소프트 보팅으로 나뉨
하드 보팅 : 다수의 분류기가 예측한 결과값을 최종 결과로 선정
소프트 보팅 :  모든 분류기가 예측한 레이블 값의 결정 확률 평균을 구한 뒤 가장 확률이 높은 레이블 값을 최종 결과로 선정

- 배깅(Bagging)
데이터 샘플링(Bootstrap)을 통해 모델을 학습시키고 결과를 집계하는 방법
모두 같은 유형의 알고리즘 기반의 분류기를 사용
데이터 분할시 중복을 허용
범주형(categorical) 데이터 : 다수결 투표 방식으로 결과 집계
연속형(continuous) 데이터 : 평균값 집계
과적합방지에 효과적
대표적이 배깅 방식 : 랜덤 포레스트 알고리즘

- 부스팅(Boosting)
여러 개의 분류기가 순차적으로 학습을 수행
이전 분류기가 예측이 틀린 데이터에 대해 올바르게 예측할 수 있도록 다음 분류기에 가중치를 부여하면서 학습과 예측을 진행
계속 분류기에 가중치를 부스팅하며 학습을 진행하기에 부스팅 방식이라고 불림
예측 성능이 뛰어나 앙상블 학습을 주도
대표적 부스팅 모듈 :  XGBoost, LightGBM
보통 부스팅 방식은 배깅에 비해 성능이 좋지만, 속도가 느리고 과적합 발생할 가능성이 존재
```

# 모델 앙상블

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input, Reshape

import time

# 회귀 문제

In [2]:
x = np.arange(-1, 1, 0.01)
np.random.shuffle(x)
y = np.sin(x)

i = int(x.shape[0]*0.8)

train_x, test_x = x[:i], x[i:]
train_y, test_y = y[:i], y[i:]


In [3]:
def build_model():  
  model = keras.Sequential()
  model.add(Input((1,)))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1))

  model.compile(optimizer="Adam", loss="mse")

  return model

In [4]:
n_members = 3

In [5]:
def build_model_A():
  model = keras.Sequential()
  model.add(Input((1,)))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1))

  model.compile(optimizer="Adam", loss="mse")
  return model

def build_model_B():
  model = keras.Sequential()
  model.add(Input((1,)))
  model.add(Dense(5, activation='relu'))
  model.add(Dense(5, activation='relu'))
  model.add(Dense(1))

  model.compile(optimizer="Adam", loss="mse")
  return model
  
def build_model_C():
  model = keras.Sequential()
  model.add(Input((1,)))
  model.add(Dense(3, activation='tanh'))
  model.add(Dense(3, activation='tanh'))
  model.add(Dense(1))

  model.compile(optimizer="Adam", loss="mse")
  return model  

In [6]:
models = []
models.append(build_model_A())
models.append(build_model_B())
models.append(build_model_C())
# for _ in range(n_members):
# 	model = build_model()
# 	models.append(model)

In [7]:
for i in range(len(models)):
	models[i].fit(train_x, train_y, epochs=1, batch_size=32)

5/5 [==============================] - 0s 4ms/step - loss: 0.1027


#### y_ = [model.predict(test_x) for model in models] 이해해 보기

In [8]:
a = [0,1,2,3,4]

b =[]
for i in a:
  b.append(i*2)

b = [i*2 for i in a]

print(b)

y_ = []
for model in models:
  y_.append(model.predict(test_x))

y_ = [model.predict(test_x) for model in models]
print(y_)

[0, 2, 4, 6, 8]
[array([[ 0.26438048],
       [-0.14709385],
       [ 0.3355059 ],
       [-0.2826294 ],
       [ 0.20580658],
       [-0.2746567 ],
       [ 0.32713822],
       [-0.2945884 ],
       [ 0.23090969],
       [ 0.32295436],
       [-0.1829709 ],
       [-0.22682065],
       [-0.23479334],
       [-0.16702552],
       [ 0.29366744],
       [-0.13114847],
       [ 0.31458664],
       [-0.254725  ],
       [-0.1152031 ],
       [ 0.30203512],
       [-0.01975177],
       [ 0.31040284],
       [ 0.40663132],
       [ 0.28111586],
       [ 0.29785123],
       [-0.3663425 ],
       [-0.39424694],
       [-0.29857472],
       [-0.3583699 ],
       [ 0.3982636 ],
       [-0.15108018],
       [ 0.0164198 ],
       [-0.2228343 ],
       [-0.19492991],
       [ 0.22672583],
       [ 0.3564251 ],
       [-0.31452012],
       [ 0.18488736],
       [-0.07533972],
       [-0.13912115]], dtype=float32), array([[ 0.30000144],
       [-0.04646055],
       [ 0.38470042],
       [-0.08291047]

In [9]:
a = [-4,-3,-2,-1,0,1,2,3,4]

# list b : a*2 +1
b= [i*2+1 for i in a]
print(b)

# list c : abs(a)
c = [abs(i) for i in a]
print(c)

def my_func(v):
  return v*3

# list d : my_func(a)
d = [my_func(i) for i in a]
print(d)

[-7, -5, -3, -1, 1, 3, 5, 7, 9]
[4, 3, 2, 1, 0, 1, 2, 3, 4]
[-12, -9, -6, -3, 0, 3, 6, 9, 12]


In [10]:
y_ = [model.predict(test_x) for model in models]
# print(y_)
print(len(y_))
print(y_[0])

print(models[0].predict(test_x))

3
[[ 0.26438048]
 [-0.14709385]
 [ 0.3355059 ]
 [-0.2826294 ]
 [ 0.20580658]
 [-0.2746567 ]
 [ 0.32713822]
 [-0.2945884 ]
 [ 0.23090969]
 [ 0.32295436]
 [-0.1829709 ]
 [-0.22682065]
 [-0.23479334]
 [-0.16702552]
 [ 0.29366744]
 [-0.13114847]
 [ 0.31458664]
 [-0.254725  ]
 [-0.1152031 ]
 [ 0.30203512]
 [-0.01975177]
 [ 0.31040284]
 [ 0.40663132]
 [ 0.28111586]
 [ 0.29785123]
 [-0.3663425 ]
 [-0.39424694]
 [-0.29857472]
 [-0.3583699 ]
 [ 0.3982636 ]
 [-0.15108018]
 [ 0.0164198 ]
 [-0.2228343 ]
 [-0.19492991]
 [ 0.22672583]
 [ 0.3564251 ]
 [-0.31452012]
 [ 0.18488736]
 [-0.07533972]
 [-0.13912115]]
[[ 0.26438048]
 [-0.14709385]
 [ 0.3355059 ]
 [-0.2826294 ]
 [ 0.20580658]
 [-0.2746567 ]
 [ 0.32713822]
 [-0.2945884 ]
 [ 0.23090969]
 [ 0.32295436]
 [-0.1829709 ]
 [-0.22682065]
 [-0.23479334]
 [-0.16702552]
 [ 0.29366744]
 [-0.13114847]
 [ 0.31458664]
 [-0.254725  ]
 [-0.1152031 ]
 [ 0.30203512]
 [-0.01975177]
 [ 0.31040284]
 [ 0.40663132]
 [ 0.28111586]
 [ 0.29785123]
 [-0.3663425 ]
 [-0.39

In [11]:
y_ = [model.predict(test_x) for model in models]
y_ = np.array(y_)
print(y_.shape)

(3, 40, 1)


In [12]:
y_ = [model.predict(test_x) for model in models]
y_ = np.array(y_)
print("y_.shape =", y_.shape)
outcomes = np.mean(y_,axis=0).squeeze()
print("outcomes.shape =", outcomes.shape)

mse = tf.keras.losses.mean_squared_error(test_y, outcomes).numpy()
print("mse =", mse)

y_.shape = (3, 40, 1)
outcomes.shape = (40,)
mse = 0.11679604


# 분류 문제

In [13]:
(raw_train_x, raw_train_y), (raw_test_x, raw_test_y) = tf.keras.datasets.mnist.load_data()

train_x = raw_train_x / 255.
test_x = raw_test_x / 255.

train_y = raw_train_y
test_y = raw_test_y

11501568/11490434 [==============================] - 0s 0us/step


In [14]:
def build_model():  
  model = keras.Sequential()
  model.add(Input((28,28)))
  model.add(Reshape((28,28,1)))
  model.add(Conv2D(32, (3, 3), padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(10, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

  return model

In [15]:
n_members = 3

In [16]:
models = []
for _ in range(n_members):
	model = build_model()
	models.append(model)

In [17]:
for i in range(n_members):
	models[i].fit(train_x, train_y, epochs=1, batch_size=32)

1875/1875 [==============================] - 8s 4ms/step - loss: 0.6480 - accuracy: 0.8045


In [18]:
from sklearn.metrics import accuracy_score

y_ = [model.predict(test_x) for model in models]
y_ = np.array(y_)  # <------------------
summed = np.sum(y_, axis=0)
print(summed[0])
outcomes = np.argmax(summed, axis=1)
print(outcomes)

acc = accuracy_score(test_y, outcomes)
print(acc)


[1.4279074e-03 3.0180857e-07 1.1702141e-03 4.1904241e-05 6.3435721e-08
 3.3178716e-07 1.3992771e-07 2.9968016e+00 2.6346220e-06 5.5487017e-04]
[7 2 1 ... 4 5 6]
0.9751


# 2진 분류 문제

In [24]:
!wget https://raw.githubusercontent.com/dhrim/MDC_2021/master/material/deep_learning/data/sonar.csv

--2022-01-19 05:12:26--  https://raw.githubusercontent.com/dhrim/MDC_2021/master/material/deep_learning/data/sonar.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87776 (86K) [text/plain]
Saving to: ‘sonar.csv’

sonar.csv           100%[===================>]  85.72K  --.-KB/s    in 0.005s  

2022-01-19 05:12:26 (15.7 MB/s) - ‘sonar.csv’ saved [87776/87776]



In [25]:
raw_data = pd.read_csv("sonar.csv")
data = raw_data.to_numpy()
np.random.shuffle(data)

x = data[:,:60]
y = data[:,60]

i = int(len(x)*0.8)
train_x, test_x = x[:i], x[i:]
train_y, test_y = y[:i], y[i:]

In [26]:
def build_model():  

  model = keras.Sequential()
  model.add(Dense(10, activation='relu', input_shape=(60,)))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1, activation="sigmoid"))

  model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["acc"])

  return model

In [27]:
n_members = 3

In [28]:
models = []
for _ in range(n_members):
	model = build_model()
	models.append(model)

In [29]:
for i in range(n_members):
	models[i].fit(train_x, train_y, epochs=1, batch_size=32, validation_split=0.2)  # <---------------

5/5 [==============================] - 1s 45ms/step - loss: 0.6949 - acc: 0.4924 - val_loss: 0.6828 - val_acc: 0.5758


In [30]:
y_ = [model.predict(test_x[:10]) for model in models]
y_ = np.array(y_).squeeze()
# print(y_)
predicted = (y_>0.5).astype(np.int)
print(predicted)

import scipy
outcomes, _ = scipy.stats.mode(predicted)
print(outcomes[:10])

[[0 0 0 1 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 0]
 [1 1 1 1 1 1 1 1 1 1]]
[[1 1 1 1 1 1 1 1 1 0]]


In [31]:
y_ = [model.predict(test_x) for model in models]
y_ = np.array(y_).squeeze()
predicted = (y_>0.5).astype(np.int)

import scipy
outcomes, _ = scipy.stats.mode(predicted)      # 넘파이 배열 모드 계산
print(outcomes[:10])
outcomes = outcomes.squeeze()


from sklearn.metrics import accuracy_score

acc = accuracy_score(test_y, outcomes)
print("acc=",acc)

[[1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1]]
acc= 0.6190476190476191
